In [11]:
import pandas as pd


# pathogenecity_type='noncoding'
# datafile='/home/sunhuaikuan/ondemand/blue_papers/DNA_LLM_REVIEW/datasets/task04-pathogenecity/clinvar_20240805.'+pathogenecity_type+'.txt'

pathogenecity_type='missense'
datafile='/home/sunhuaikuan/ondemand/blue_papers/DNA_LLM_REVIEW/datasets/task04-pathogenecity/clinvar_20240805.'+pathogenecity_type+'_matched.txt'

# csv_filename = '/home/sunhuaikuan/ondemand/blue_papers/DNA_LLM_REVIEW/preprocess/pathogenecity/pathogenecity_caduceus_noncoding.csv'

df_origin=pd.read_csv(datafile, delimiter='\t')
columns_to_keep=['CHROM','POS','Pathogenicity'] # 'ID','REF','ALT',
df_origin = df_origin[columns_to_keep]
df_origin

/scratch/local/48790182/ipykernel_3131455/1957555159.py:12: DtypeWarning: Columns (0,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_origin=pd.read_csv(datafile, delimiter='\t')


,CHROM,POS,Pathogenicity
0,1,1020176,B
1,1,1020177,B
2,1,1020185,B
3,1,1020195,B
4,1,1020198,B
...,...,...,...
560183,Y,2787548,B
560184,Y,2787549,B
560185,Y,2787580,B
560186,Y,2787582,B


## Process original data

In [12]:
df = df_origin

for i in range(1,23):
    df.loc[df['CHROM']==i,'CHROM']=str(i)

df=df[~df['CHROM'].isna()]
df = df[~df['CHROM'].str.contains('KI')]
df = df[~df['CHROM'].str.contains('GL')]

max_length= 128 # 186

df['START']= df['POS']- max_length //2 -1
df['END']  = df['START'] + max_length
# df['size'] = max_length

Pathogenicity_dict={'B':0,'P':1}
df['y'] = df['Pathogenicity'].map(Pathogenicity_dict)

df.drop(['POS', 'Pathogenicity'], axis=1, inplace=True)

dataframe = df

dataframe

,CHROM,START,END,y
0,1,1020111,1020239,0
1,1,1020112,1020240,0
2,1,1020120,1020248,0
3,1,1020130,1020258,0
4,1,1020133,1020261,0
...,...,...,...,...
560183,Y,2787483,2787611,0
560184,Y,2787484,2787612,0
560185,Y,2787515,2787643,0
560186,Y,2787517,2787645,0


In [13]:
from Bio import SeqIO

sequences = list(SeqIO.parse("/blue/xiaofan/chenyuanhan/data/genome.hg38rg.fa", "fasta"))

def extract_sequence_segment(seq_id, start, end, sequences):
    for seq_record in sequences:
        if seq_record.id == seq_id:
            segment = str(seq_record.seq[start:end])
            return segment
    return None

In [14]:
rows = []
for index, row in dataframe.iterrows():
    chrom=str(row['CHROM'])
    start=int(row['START'])
    end=int(row['END'])
    y=row['y']
    # rowid=row['ROWID']
    segment = extract_sequence_segment(chrom, start, end, sequences)
    rows.append([segment,  y]) # rowid,

columns=['sequence','y']  # 'rowid',
df = pd.DataFrame(rows, columns=columns)
# df = df.drop(columns=['rowid','y'])
df.to_csv('dna_segment_'+pathogenecity_type+'.csv', index=False)
df

,sequence,y
0,TCCTGTCCAGTCCCGTCCCCGGCGCGGCCCGCGCGCTCCTCCGCCG...,0
1,CCTGTCCAGTCCCGTCCCCGGCGCGGCCCGCGCGCTCCTCCGCCGC...,0
2,GTCCCGTCCCCGGCGCGGCCCGCGCGCTCCTCCGCCGCCTCTCGCC...,0
3,CGGCGCGGCCCGCGCGCTCCTCCGCCGCCTCTCGCCTGCGCCATGG...,0
4,CGCGGCCCGCGCGCTCCTCCGCCGCCTCTCGCCTGCGCCATGGCCG...,0
...,...,...
560183,ACAGCTTTCAGTGCAAAGGAAGGAAGAGCTTCTCCGGAGAGCGGGA...,0
560184,CAGCTTTCAGTGCAAAGGAAGGAAGAGCTTCTCCGGAGAGCGGGAA...,0
560185,TCCGGAGAGCGGGAATATTCTCTTGCACAGCTGGACTGTAATCATC...,0
560186,CGGAGAGCGGGAATATTCTCTTGCACAGCTGGACTGTAATCATCGC...,0
